Import of Brightway to build coding-enviroment.

In [30]:
import bw2io as bi
FILEPATH = "lca-epe-databases.tar.gz"
bi.restore_project_directory(fp=FILEPATH, project_name="LCA_EPE", overwrite_existing=True)


Restoring project backup archive - this could take a few minutes...
Restored project: LCA_EPE


'LCA_EPE'

Import of bio- and technosphere.

In [31]:
import bw2data as bd  # for everything related to the database
import bw2calc as bc  # for the actual LCA calculations

Your LCA processes live in databases, and your databases live in projects. You need to select a project when working with brightway. we will list all projects below, and then set the current project.

In [32]:
bd.projects  # if this list does not include LCA_EPE, please take another look at the welcome.ipynb notebook
bd.projects.set_current("LCA_EPE")
bd.databases

Databases dictionary with 4 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ei310_IMAGE_SSP2_RCP26_2035
	ei310_IMAGE_SSP2_RCP26_2050

We have pre-installed the ecoinvent database, a biosphere database, and some other databases that you do not need to worry about yet. The biosphere database includes all elementary flows to and from the environment. The ecoinvent database contains inventory information on technical processes. To access them easily later, we give them nice and short names:

In [33]:
# definition of the databases
eidb = bd.Database("ecoinvent-3.10-cutoff")
bsdb = bd.Database("ecoinvent-3.10-biosphere")

Here we search for processes with terms related to syngas.

In [34]:
# Search terms related to syngas
syngas_terms = [
    "syngas production",
    "gasification",
    "biomass gasification",
    "steam reforming",
    "coal-to-liquid",
    "biomass-to-liquid",
    "CO2-based production",
    "carbon dioxide utilization",
    "methane reforming",
    "hydrogen production",
    "carbon monoxide production"  # Added term
]

# Search terms related to Fischer-Tropsch
ft_terms = [
    "Fischer-Tropsch",
    "Tropsch",
    "synthetic kerosene",
    "synthetic hydrocarbons",
    "gas-to-liquid",
    "synthetic fuels",
    "hydrocarbon synthesis",
    "hydrocracking",
    "distillation",
    "combined heat and power",
    "electricity for Fischer-Tropsch",
    "renewable electricity",
    "naphtha",
    "jet fuel"
]

# Search for syngas-related processes
syngas_results = []
syngas_no_matches = []  # List to store terms with no matches
for term in syngas_terms:
    matches = [act for act in eidb if term in act["name"]]
    if matches:
        syngas_results.extend(matches)
    else:
        syngas_no_matches.append(term)

# Search for Fischer-Tropsch-related processes
ft_results = []
ft_no_matches = []  # List to store terms with no matches
for term in ft_terms:
    matches = [act for act in eidb if term in act["name"]]
    if matches:
        ft_results.extend(matches)
    else:
        ft_no_matches.append(term)

# Remove duplicates from the results
syngas_results = list(set(syngas_results))
ft_results = list(set(ft_results))

# Print syngas-related results
print("Syngas-related processes:")
for result in syngas_results:
    print(result)

# Print syngas terms with no matches
print("\nSyngas terms with no matches:")
for term in syngas_no_matches:
    print(term)

# Print Fischer-Tropsch-related results
print("\nFischer-Tropsch-related processes:")
for result in ft_results:
    print(result)

# Print Fischer-Tropsch terms with no matches
print("\nFischer-Tropsch terms with no matches:")
for term in ft_no_matches:
    print(term)


Syngas-related processes:
'ammonia production, steam reforming, liquid' (kilogram, RU, None)
'hydrogen production, steam methane reforming' (kilogram, US, None)
'hydrogen production, steam methane reforming' (kilowatt hour, RoW, None)
'ammonia production, steam reforming, liquid' (kilogram, UN-OCEANIA, None)
'methanol production, coal gasification' (kilogram, RoW, None)
'hydrogen production, gaseous, petroleum refinery operation' (kilogram, Europe without Switzerland, None)
'hydrogen production, gaseous, petroleum refinery operation' (kilogram, CO, None)
'ammonia production, steam reforming, liquid' (kilogram, RoW, None)
'hydrogen production, steam methane reforming' (kilowatt hour, CN, None)
'hydrogen production, gaseous, petroleum refinery operation' (kilogram, PE, None)
'methanol production, biomass gasification' (kilogram, CH, None)
'hydrogen production, coal gasification' (kilogram, CN, None)
'hydrogen production, steam methane reforming' (kilowatt hour, US, None)
'hydrogen produc

In [35]:
# let's refine our search by using the location
[
    act
    for act in eidb
    if "synthetic fuel production, from coal, high temperature Fisher-Tropsch operations" in act["name"] and "ZA" in act["location"]
]

['synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None),
 'synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None),
 'synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None),
 'synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None),
 'synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None),
 'synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None),
 'synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None),
 'synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None),
 'synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None),
 'synthetic fuel production, from coal, high t

In [36]:
Fischer_Tropsch1 = bd.get_node(database="ecoinvent-3.10-cutoff", name="synthetic fuel production, from coal, high temperature Fisher-Tropsch operations", location="ZA", **{"reference product": "heavy fuel oil"})

In [37]:
Fischer_Tropsch1.as_dict()

{'comment': 'This dataset represents the synthetyc fuels production from coal in South Africa (ZA). These operations are supported by mining coal and converting it into synthetic fuels and chemicals through proprietary Fischer-Tropsch technology by Sasol, the biggest synfuels producer in ZA (and worldwide). Its productions accounts for 150 000 bbl/day of crude-equivalent. \nHigh Temperature Fisher-Tropsch process makes use of an iron-based catalyst. However no information on catalyst supply or the amount used was available in the public reports.\nGeography:  SECUNDA SYFUEL OPERATIONS are  bases in Secunda (South Africa); SSO receives coal from 5 mines in Mpumalanga province. SECUNDA CHEMICAL OPERATIONS operates production facilities in Secunda (SA). Attached figure show the Secunda complex (inclusive of the mining sites) and the operation based in Secunda and Sasolburg.\nImage: https://db3.ecoinvent.org/images/a79dc0c2-0dda-47ec-94e0-6f076bc8cdb6\nTechnology:  SECUNDA SYNFUEL OPERATION

In [39]:
list(Fischer_Tropsch1.exchanges())

[Exchange: 1.0 kilogram 'synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None) to 'synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None)>,
 Exchange: 0.12382492849560624 kilogram 'market for chemical, inorganic' (kilogram, GLO, None) to 'synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None)>,
 Exchange: 0.21929833033485835 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, ZA, None) to 'synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None)>,
 Exchange: 1.2313392566973338 kilogram 'market for hard coal' (kilogram, ZA, None) to 'synthetic fuel production, from coal, high temperature Fisher-Tropsch operations' (kilogram, ZA, None)>,
 Exchange: -0.3521632933486245 kilogram 'market for hard coal ash' (kilogram, RoW, None) to 'synthetic fuel production, from coal, high temperatur